In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from keras import layers

Using TensorFlow backend.


In [4]:
from keras.layers import RepeatVector,Concatenate,Dense,Activation

In [5]:
import numpy as np

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
num_words=399965

In [8]:
ds_path='/home/al/Desktop/nnflproj/glove.6B.100d.txt'
data = np.loadtxt(ds_path,dtype=str,usecols=0)
d2 = np.loadtxt(ds_path,dtype=float,usecols=range(1,101))

d=data.reshape([399965,1])
d=np.concatenate([d,d2],axis=1)


In [9]:
word_to_ix=dict(zip(data,range(0,num_words+1)))
ix_to_word=dict(zip(range(0,num_words+1),data))

In [84]:
Xa=keras.layers.Input(shape=(50,100))
lstm=keras.layers.LSTM(50,return_sequences=True)
bi=keras.layers.Bidirectional(lstm)
a=bi(Xa)
a=bi(a)
model=keras.Model(inputs=Xa,outputs=a)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 50, 100)]    0                                            
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, 50, 100)      60400       input_10[0][0]                   
                                                                 bidirectional_8[0][0]            
Total params: 60,400
Trainable params: 60,400
Non-trainable params: 0
__________________________________________________________________________________________________


In [85]:

model.compile( loss='cosine_similarity', metrics=['accuracy'])

In [83]:
d_path='/home/al/Desktop/nnflproj/nds/nds-a'

for e  in 'a':
#bcdefghi':
    for f  in 'abcdefghijklmnopqrstuvwxyz':
        ds = np.loadtxt(d_path+e+f+'.csv',delimiter='\n',dtype='str')
        X=np.ndarray(shape=[ds.shape[0],50,100])
        y=np.ndarray(shape=[ds.shape[0],50])
        for j in range(1,ds.shape[0]-(ds.shape[0]%1000)+1):
           
            w1=np.array(ds[j-1].lower().split())
            w1_sum=np.zeros([100,])
            for i in range(0,w1.shape[0]):    
                
                
                w=w1[i]
                if i==49:
                    #w1_sum/=50
                    X[j,i,:]=d2[word_to_ix['.']]#for<end>
                    y[j,i]=word_to_ix['.']#for<end>
                    break
                if w=='<start>':
                    continue
                if w=='<end>':
                    #w1_sum/=len(w1)
                    while i<49:
                        X[j,i,:]=d2[word_to_ix[',']]
                        y[j,i]=word_to_ix[',']
                        i+=1
                    oh[num_words]=1
                    X[j,i,:]=d2[word_to_ix['.']]#for<end>
                    y[j,i]=word_to_ix['.']
                    break
                try:                
                    #w1_sum+=d2[word_to_ix[w]]
                    
                    X[j,i,:]=d2[word_to_ix[w]]
                    y[j,i]=word_to_ix[w]
                except KeyError:
                    #deal with unknown words
                    
                    X[j,i,:]=d2[word_to_ix['alfred']]#for<unk>
                    y[j,i]=word_to_ix['alfred']
                    continue
            if(j-1%1000==0&&j!=0):
                    model.fit(X[1001-j:j-1,:,:],X[1000-j:j,:,:],epoch=1,batch_size=1000)
            
            #np.append(df,w1_sum)
        #model.fit(np,epochs=1,batch_size=ds.shape[0]+1)

In [86]:
X1=X[0:9999,:,:]
X2=X[1:10000,:,:]

In [87]:
model.fit(X1,X2,epochs=1,batch_size=1000)
#, batch_size=ds.shape[0])


Train on 9999 samples
9999/9999 [==============================] - 125s 13ms/sample - loss: -0.6599 - accuracy: 0.0049


In [79]:
model.predict()

ValueError: Error when checking input: expected input_9 to have 3 dimensions, but got array with shape ()